In [1]:
import pandas as pd
import numpy as np
import json
import os

import warnings
warnings.filterwarnings("ignore")

In [17]:
os.chdir(r"D:\A李杜若\留学\科研\可视化_王懿芳学姐\CFPS\NLS\NLSW")

In [18]:
d = {}
d['person_id']=''
d['basic_info']={"gender":"",
                 "age":0, #birth_year
                  "race":""}

d["education"] = { 
          "year":0,
          "highest_education_rank":""
        }

d["life_sequence"] =  []
ls = { "year":0,
    "marriage_status":"",
    "marriage_status_change":False,
    "childbearing_status":True,
    "child_under_18":True,
    'child_under_18_status_change':None, #0,1,2
    "residence":"",
    "migration_status":True,
    
    "isWorking":True,
    "income":0,
    "occupation":0
  }

d["spouse_info"]= {
  "spouse_id":"",
  "spouse_age":0,
  "spouse_race":"",
  "spouse_education": {
    "year",
    "highest_education_rank",
  },
  "spouse_first_job": {
    "year":0,
    "income":0,
    "occupation":0,
  },
  "spouse_life_sequence":[]
}

## input

In [19]:
basic_info = pd.read_csv("./basic_info.csv")
edu_info = pd.read_csv("./edu_info.csv")
job_info = pd.read_csv("./job_info.csv")
mar_info = pd.read_csv("./marry_info.csv")
res_info = pd.read_csv('./res_info.csv')
childbirth_info = pd.read_csv('./childbirth_year_info.csv')
#first_year(69); 68,69 if same?
first_job_info = pd.read_csv('./first_job_info.csv')
## spouse info
# 68's husband birth year
sp_bir_info = pd.read_csv('./spouse_info/sp_bir_info.csv')
# husband's id in NLS-Men
sp_id_info = pd.read_csv('./spouse_info/spid_in_NLSM.csv')
#income and occu
sp_inc_info = pd.read_csv('./spouse_info/income_Y_sp.csv')
sp_occu_info = pd.read_csv('./spouse_info/occu_Y_sp.csv')


In [20]:
df_id = basic_info[["R0000100"]]

In [21]:
n = basic_info.shape[0] #5159 people (68-03 -> 22 interview years)

In [22]:
# create dictionary list
d = {}
d['person_id']=''
d['basic_info']={
    "age":0, #birth_year
    "race":"",

    "education": { 
        "year":0,
        "highest_education_level": "", 
        "highest_education_name": "",
        },
    
    "first_job": {
        "year":0,
        "income":0,
        "occupation":0,
        }
    }
d["life_sequence"] =  []
ls = { "year":0,
    "marriage_status":"",
    "marriage_status_change":False,
    "childbearing_status":True,
    "child_under_18":True,
    'child_under_18_status_change':None,
    "residence":"",
    "migration_status":True,
    
    "isWorking":True,
    "income":0,
    "occupation":0
}

d["spouse_info"]= {
"spouse_id":"",
"spouse_age":0,
"spouse_race":"",
"spouse_education": {
    "year":0,
    "highest_education_level": "", 
    "highest_education_name": "",
},
"spouse_first_job": {
    "year":0,
    "income":0,
    "occupation":0,
},
"spouse_life_sequence":[]
}

people_dict = [{"person_id": str(i+1), 'basic_info': {}, 'life_sequence': [], 'spouse_info': {}} for i in range(n)]

## gender, birth, race

In [23]:
#1. gender, birth_year, race

race_map = {
    1: 'white',
    2: 'black',
    3: 'other'
}
for idx, person in enumerate(people_dict):
    year = basic_info['R0042012'][idx]
    race = basic_info['R0003200'][idx]
    
    if year < 0: 
        people_dict[idx]['basic_info']['birth_year'] = None
    else:
        people_dict[idx]['basic_info']['birth_year'] = int(year + 1900)
    if race < 0: 
        people_dict[idx]['basic_info']['race'] = None
    else:
        people_dict[idx]['basic_info']['race'] = race_map[race]

## education

In [24]:
#education
#'R7611000':highest_grade completed(2003-revised)
#'R7611000-l': highest_degree_name (created)
edu_info[['R7611000-l']]=None
for i in range(n):
    if 12<=edu_info['R7611000'][i]<=15:
        edu_info['R7611000-l'][i]= "high-school"
    elif 16<=edu_info['R7611000'][i]<=18:
        edu_info['R7611000-l'][i]= "college"
    elif (1<=edu_info['R7611000'][i]<=11):
        edu_info['R7611000-l'][i]= "high-school-noncompletion" 
   

In [26]:
# the year got the highest grade
# YEAR R RECEIVED HIGH SCHOOL (GED) DIPLOMA, 83; 19xx
high_school_code_list1 = ['R0927800']
# High_school:Have you received a high school diploma or GED since DATE OF LAST INTERVIEW? 
# 1: yes vs 0: no
high_school_code_list2 = ['R1051800', 'R1097600', 'R1215400', 'R1346700', 'R1520700', 'R3477200', 'R4193400', 'R5142100', 'R6236200', 'R7337400']
high_school_year = [1985,1987,1988,1991,1993,1995,1997,1999,2001,2003]
# DATE R RECEIVED HIGHEST COLLEGE DEGREE, - YEAR; 
# idx<=5 位数+1900
college_code_list = ['R0591700','R1052400','R1098200','R1216000','R1347200','R1521300','R3477800','R4194000', 'R5142700', 'R6236800', 'R7338000'] 

high_school_map = dict(zip(high_school_code_list2, high_school_year))


index = [0]+[i for i in range(1,19)]
name = [None]+['1st grade','2nd grade','3rd grade']+[str(i)+'th grade' for i in range(4,13)]+['1st year college','2nd year college','3rd year college']+[str(i)+'th year college' for i in range(4,6)]+['6th year college or more']
degree_map = dict(zip(index,name))

In [27]:
for idx, person in enumerate(people_dict):
    edu_dict = {'year': None, 'highest_education_level': None, 'highest_education_name': None}
    if edu_info['R7611000'][idx]>0: #有最高学位数据
# high_school grade        
        if edu_info['R7611000-l'][idx]== "high-school":
            if edu_info['R0927800'][idx] > 0:
                edu_dict['year'] = int(edu_info['R0927800'][idx]+1900)
            else:
                for h_idx, h_year in enumerate(high_school_code_list2):
                    if edu_info[h_year][idx]==1:
                        edu_dict['year'] = int(high_school_map[h_year]+1900)
                    break
# college degree
        if edu_info['R7611000-l'][idx]== "college":
                  for c_idx, c_var in enumerate(college_code_list):
                      if edu_info[c_var][idx] > 0:
                          if c_idx <=5:
                              edu_dict['year'] = int(edu_info[c_var][idx]+1900)
                          else:
                              edu_dict['year'] = int(edu_info[c_var][idx])
# name and level                            
        edu_dict['highest_education_name'] = degree_map[edu_info['R7611000'][idx]]
        edu_dict['highest_education_level'] = edu_info['R7611000-l'][idx]
    person['basic_info']['education'] = edu_dict
    

## first_job

In [28]:
occu_code_list = ['R0018700', 'R0135800', 'R0220100', 'R0327000', 'R0410700',
       'R0495200', 'R0544300', 'R0585400', 'R0702600', 'R0754200', 'R0801300',
       'R0944000', 'R1060400', 'R1106400', 'R1230800', 'R1362400', 'R1579900',
       'R1902900', 'R3640200', 'R4442700', 'R5640600', 'R6718100']

inc_code_list = ['R0112200', 'R0192000', 'R0307600', 'R0391400', 'R0474000',
       'R0532200', 'R0581700', 'R0671200', 'R0746500', 'R0793200', 'R0902100',
       'R1045500', 'R1091600', 'R1196300', 'R1328500', 'R1506500', 'R3388500',
       'R4153500', 'R5111700', 'R6204300', 'R7312300', 'income_2003']
year_list = [1968,1969,1970,1971	,1972	,1973	,1975	,1977	,1978	,1980	,1982	,1983	,1985	,1987	,1988	,1991	,1993	,1995	,1997,1999,2001,2003]

for idx, person in enumerate(people_dict):
    first_job_dict = {'year': None, 'income': None, 'occupation': None}
    # 1969 之前的 first job
    if first_job_info['R0099500'][idx] > 0:
        first_job_dict['year'] = int(first_job_info['R0099500'][idx]+1900)
        #如果是68，69年 first_job，直接对应 【occu, income】
        #如果<68，根据两个对应变量（是否1st job和68，69的job一致）
        if (first_job_info['R0099500'][idx] == 68 or first_job_info['R0024500'][idx] == 1) and (job_info[occu_code_list[0]][idx] > 0 or job_info[inc_code_list[0]][idx] > 0):
            first_job_dict['income'] = int(job_info[inc_code_list[0]][idx])
            first_job_dict['occupation'] = int(job_info[occu_code_list[0]][idx])
        elif  (first_job_info['R0099500'][idx] == 69 or first_job_info['R0099300'][idx] == 1) and (job_info[occu_code_list[1]][idx] > 0 or job_info[inc_code_list[1]][idx] > 0):  
            first_job_dict['income'] = int(job_info[inc_code_list[1]][idx])
            first_job_dict['occupation'] = int(job_info[occu_code_list[1]][idx])
    else:
        for y_idx, year in enumerate(year_list):
            occu = job_info[occu_code_list[y_idx]][idx]
            income = job_info[inc_code_list[y_idx]][idx]
            if occu > 0 or income > 0:
                first_job_dict['year'] = int(year)
                if occu > 0:
                    first_job_dict['occupation'] = int(occu)
                if income > 0:
                    first_job_dict['income'] = int(income)
                break       
    person['basic_info']['first_job'] = first_job_dict

## life_sequence

In [29]:
res_code_list = ['R0003306', 'R0086206', 'R0146106', 'R0253306', 'R0333606',
       'R0415506', 'R0517906', 'R0546916', 'R0586106', 'R0753400', 'R0800600',
       'R0943300', 'R1056800', 'R1105700', 'R1229400', 'R1361500', 'R1579000',
       'R3492100', 'R4235600', 'R5407300', 'R6318000', 'R7610300']

mar_status_code_list = ['R0042051', 'R0116651', 'R0197050', 'R0311951', 'R0396251',
       'R0480551', 'R0519600', 'R0581000', 'R0635000', 'R0710000', 'R0756500',
       'R0940700', 'R0947400', 'R1063000', 'R1221000', 'R1352700', 'R1569900',
       'R1664710', 'R3507200', 'R4278200', 'R5447500', 'R6516200']

occu_code_list = ['R0018700', 'R0135800', 'R0220100', 'R0327000', 'R0410700',
       'R0495200', 'R0544300', 'R0585400', 'R0702600', 'R0754200', 'R0801300',
       'R0944000', 'R1060400', 'R1106400', 'R1230800', 'R1362400', 'R1579900',
       'R1902900', 'R3640200', 'R4442700', 'R5640600', 'R6718100']

inc_code_list = ['R0112200', 'R0192000', 'R0307600', 'R0391400', 'R0474000',
       'R0532200', 'R0581700', 'R0671200', 'R0746500', 'R0793200', 'R0902100',
       'R1045500', 'R1091600', 'R1196300', 'R1328500', 'R1506500', 'R3388500',
       'R4153500', 'R5111700', 'R6204300', 'R7312300', 'income_2003']
#1st-16th child
chb_code_list = ['R7367100', 'R7367200', 'R7367300', 'R7367400', 'R7367500','R7367600', 'R7367700', 'R7367800', 'R7367900', 'R7368000', 'R7368100','R7368200', 'R7368300', 'R7368400', 'R7368500', 'R7368600'] 

year_list = [1968,	1969, 1970, 1971 ,1972,1973	,1975	,1977	,1978	,1980	,1982	,1983	,1985	,1987	,1988	,1991	,1993	,1995	,1997	,1999	,2001	,2003]

mar_map = {
       1: 'married spouse present',
       2: 'married spouse absent',
       3: 'widowed',
       4: 'divorced',
       5: 'separate',
       6: 'never married',
       7: 'spouse absent, reason unknown',
}

res_map = {
    0: 'non-south',
    1: 'south',
    9: None # Code 9 represents an incomplete interview and should be treated as an invalid
}

# childbirth_year_map: 从1994开始，直到下一个调查年之前，都算做前一个调查年，如1995年的childbirth算作1994年，这样它会被算入对1996年的影响
key_y = [i for i in range(1968,2004)]
value_y = [i for i in range(1968,1974)]+[1973,1975,1975,1977,1978,1978,1980,1980,1982,1983,1983,1985,1985,1987,1988,1988,1988,1991,1991,1993,1993,1995,1995,1997,1997,1999,1999,2001,2001,2003]
childbirth_map = dict(zip(key_y,value_y))

In [30]:
for idx, person in enumerate(people_dict):
    person['life_sequence']=[]
# get childbirth_year data raw list & processed list
    child_list = childbirth_info.iloc[idx,1:]
    child_year_list0 = list(filter(lambda x: (x>0),child_list))
    child_year_list1 = child_year_list0.copy()
    for ch_idx, ch_year in enumerate(child_year_list0):
           if ch_year>=1968:
                  child_year_list1[ch_idx] = childbirth_map[ch_year]
    

    for y_idx, year in enumerate(year_list):
       year_dict = {'year': year, 'marriage_status': None, 'marriage_status_change': None, 'childbearing_status': None, 'child_under_18': None, 'child_under_18_status_change': None, 'residence': None, 'migration_status': None, 'isWorking': None, 'occupation': None, 'income': None 
       }
# marital status
       marital_status = mar_info[mar_status_code_list[y_idx]][idx]
       if marital_status > 0:
              year_dict['marriage_status'] = mar_map[marital_status]
#marital_status_change
       if y_idx == 0:
              year_dict['marriage_status_change'] = -1
       if y_idx > 0 and person['life_sequence'][y_idx - 1]['marriage_status'] != None and year_dict['marriage_status'] != None:
              if person['life_sequence'][y_idx - 1]['marriage_status'] != year_dict['marriage_status']:
                     year_dict['marriage_status_change'] = True
              else: 
                     year_dict['marriage_status_change'] = False
# childbearing_status
       if len(child_year_list0) > 0:
              if year in child_year_list1:
                     year_dict['childbearing_status'] = True
              else:
                     year_dict['childbearing_status'] = False
# child_under_18_status
       if len(child_year_list0) > 0:
             before_year_list = list(filter(lambda x: (x <= year),child_year_list0)) 
             if len(before_year_list)==0:
                    year_dict['child_under_18'] = False
             elif 0<=(year-max(before_year_list))< 18:
                    year_dict['child_under_18'] = True
             else:
                    year_dict['child_under_18'] = False
#child_under_18_status_change
# 0: 状态不变；
# 1: 有小孩(<18岁)--> 无小孩；
# 2: 无小孩-->有小孩
       if y_idx == 0:
              year_dict['child_under_18_status_change'] = -1
       if y_idx > 0 and (person['life_sequence'][y_idx - 1]['child_under_18'] != None) and (year_dict['child_under_18'] != None):
              if person['life_sequence'][y_idx - 1]['child_under_18'] == year_dict['child_under_18']:
                     year_dict['child_under_18_status_change'] = 0
              if person['life_sequence'][y_idx - 1]['child_under_18'] == True and year_dict['child_under_18'] == False:
                     year_dict['child_under_18_status_change'] = 1
              if person['life_sequence'][y_idx - 1]['child_under_18'] == False and year_dict['child_under_18'] == True:
                     year_dict['child_under_18_status_change'] = 2          
# residence
# south, non-south 
       residence = res_info[res_code_list[y_idx]][idx]
       if residence >= 0:
              year_dict['residence'] = res_map[residence]
            
# migration_status
       if y_idx == 0:
              year_dict['migration_status'] = -1
       if y_idx > 0 and person['life_sequence'][y_idx - 1]['residence'] != None and year_dict['residence'] != None:
              if person['life_sequence'][y_idx - 1]['residence'] != year_dict['residence']:
                     year_dict['migration_status'] = True
              else:
                     year_dict['migration_status'] = False
## job information
       occu = job_info[occu_code_list[y_idx]][idx]
       income = job_info[inc_code_list[y_idx]][idx]
# isworking 
       if occu > 0 or income > 0:
              year_dict['isWorking'] = True
       else:
              year_dict['isWorking'] = False
# occupation
       if occu > 0:
              year_dict['occupation'] = int(occu)
# income
       if income > 0:
              year_dict['income'] = int(income)  
              
## 逐个加入个人的life_sequence              
       person['life_sequence'].append(year_dict)        

## spouse_info

In [31]:
sp_occu_code_list = ['R0064100', 'R0132200', 'R0211500', 'R0325550', 'R0413300',
       'R0499100', 'R0532030', 'R0581530', 'R0676440', 'R0910600', 'R1531500',
       'R2756000', 'R3961300', 'R4857100', 'R6021600', 'R7132500']
sp_inc_code_list = ['R0036500', 'R0112700', 'R0192500', 'R0308100', 'R0391900',
       'R0474500', 'R0532300', 'R0581800', 'R0672100', 'R0747600', 'R0794500',
       'R0904400', 'R1047000', 'R1093100', 'R1198600', 'R1330900', 'R1510500',
       'R3392300', 'R4157300', 'R5115700', 'R6208200', 'R7316200']

for idx, person in enumerate(people_dict):
    person['spouse_info']={
        "spouse_id":None,
        "spouse_birth_year":None,
        "spouse_race":None,
        "spouse_education": None,
        "spouse_first_job": None,
        "spouse_life_sequence":[]
    }
    
    sp_pid = sp_id_info['R0000350'][idx]
    sp_birth_year = sp_bir_info['R0042015'][idx]#1968
    
## id, birth_year, 无race
    if sp_pid > 0:
        person['spouse_info']['spouse_id'] = int(sp_pid)
    if sp_birth_year > 0:
        person['spouse_info']['spouse_birth_year'] = int(sp_birth_year+1900)
## life_sequence: occupation and income
    f = 0 #若出现1次occu不存在的年份，y_idx-1，以此累加
    for y_idx, year in enumerate(year_list):
       year_dict = {'year': year, 'childbearing_status': None, 'child_under_18': None, 'child_under_18_status_change': None, 'residence': None, 'migration_status': None, 'isWorking': None, 'occupation': None, 'income': None 
       }
# job information
       if year in [1980,1982,1985,1987,1988,1991]:
              occu = -1
              f = f+1
       else:
              occu = sp_occu_info[sp_occu_code_list[y_idx-f]][idx]
       income = sp_inc_info[sp_inc_code_list[y_idx]][idx]
# isworking 
       if occu > 0 or income > 0:
              year_dict['isWorking'] = True
       else:
              year_dict['isWorking'] = False
# occupation
       if occu > 0:
              year_dict['occupation'] = int(occu)
# income
       if income > 0:
              year_dict['income'] = int(income)  
              
## 逐个加入个人的life_sequence              
       person['spouse_info']['spouse_life_sequence'].append(year_dict) 

## *预处理：合并occu+income处理

In [4]:
df_occu = pd.read_csv('./occu_info.csv')
df_inc = pd.read_csv('./income_info.csv')
df_inc = df_inc.drop('R0036000',axis=1)
df_inc['income_2003']=df_inc.iloc[:,-1]
c = pd.merge(df_occu,df_inc, on='R0000100', how='outer').fillna(-1)
c.to_csv("./job_info.csv",index=False)


In [31]:
df_inc = pd.read_csv('./income_info.csv')
df_inc1 = df_inc.drop_duplicates(subset="R0000100", keep='first', inplace=False)
df_inc1 = df_inc1.drop('R0155400',axis=1)
df_inc1['income_2018']=df_inc1.iloc[:,-1]
c = pd.merge(df_occu,df_inc1, on='R0000100', how='outer').fillna(-1)
c.to_csv("./job_info.csv",index=False)

## output

In [312]:
##转化np值为原始值 e.g. int64 -> int
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        elif isinstance(obj, np.bool_):
            return super().encode(bool(obj))
        else:
            return super(NpEncoder, self).default(obj)

class CustomJSONizer(json.JSONEncoder):
    def default(self, obj):
        return super().encode(bool(obj)) \
            if isinstance(obj, np.bool_) \
            else super().default(obj)

In [16]:
with open("./NLSW_clean.json", 'w') as f:
    json.dump(people_dict,f, indent = 2, sort_keys=True)

## Explore

### 婚姻情况变化

In [32]:
##婚姻状况不变 vs 变化
mar_status_change_sum = {
    'not change': 0,
    'change': 0,
    'None': 0
}
i=k=j=0
for idx, person in enumerate(people_dict):
    mar_ch_list = []
    y_n = len(person['life_sequence'])
    for y_idx, year in enumerate(person['life_sequence']):
        mar_ch = year['marriage_status_change']
        if mar_ch != None:
                mar_ch_list.append(mar_ch)
    if len(mar_ch_list)<y_n:
        k=k+1
    elif True in mar_ch_list:
        j=j+1
    else:
        i=i+1
    mar_status_change_sum['change']=i
    mar_status_change_sum['not change']=j
    mar_status_change_sum['None']=k

In [33]:
mar_status_change_sum # 77.0% not change

{'not change': 1323, 'change': 396, 'None': 3440}

### 计算Event数目

In [34]:
indice = list(range(len(year_list)))
year_value = year_list
idx_year_map = dict(zip(indice,year_value))
event_vs_career_list = [] #每个人的event dict 的 list

for idx, person in enumerate(people_dict):
    total_num = 0
    event_dict = {
        'person_id': None,
        'first_occu_year':None,
        'last_occu_year':None,
        'first_income_year':None,
        'last_income_year':None,
        'event_n': None,
# before 1st job:0
# after 1st job:1
        "marriage_status_change":{
            'n':None,
            'happen_time':None
        }, 
        "childbearing_status":{
            'n':None,
            'happen_time':None
        },
        'child_under_18_status_change':{
            'n':None,
            'happen_time':None
        },
        'migration_status':{
            'n':None,
            'happen_time':None
        },
    }
# take person who either has occu or income information
# occu 和 income 非空index list (可以map到年份list)
    df = pd.DataFrame.from_dict(person['life_sequence'])
    df = df.replace({np.nan: None})
    occu_idx_list = [i for i, value in enumerate(list(df['occupation'])) if value != None]
    income_idx_list = [i for i, value in enumerate(list(df['income'])) if value != None]
    if len(occu_idx_list)>0 or len(income_idx_list)>0:
        event_dict['person_id'] = person['person_id']
        if len(occu_idx_list)>0:
            event_dict['first_occu_year'] = idx_year_map[occu_idx_list[0]]
            event_dict['last_occu_year'] = idx_year_map[occu_idx_list[-1]]
        if len(income_idx_list)>0:
            event_dict['first_income_year'] = idx_year_map[income_idx_list[0]]
            event_dict['last_income_year'] = idx_year_map[income_idx_list[-1]]
    
    # status_change events(marriage, child_u18, migration)
    # childbearing 没有-1
    # len(!=0)-1>=1, since first is -1
        event_name_list = ["marriage_status_change", "childbearing_status", 'child_under_18_status_change', "migration_status"]
        for e_idx, event in enumerate(event_name_list):
            event_list = df[event]
            happen_list = list(filter(None, event_list))
            if (-1 in happen_list):
                happen_list.remove(-1)
            num = len(happen_list)
            if num > 0:
                event_dict[event]['n'] = num
                total_num = total_num + num
            # get all index of True
                y_idx_list = [i for i,val in enumerate(event_list) if (val!=None and val!=0 and val!=-1)]
            # < 1st job year:'before' vs >= 1st job year:'after'
                comp_list = []
                
                
                for y_idx in y_idx_list:
                    event_year = idx_year_map[int(y_idx)]
                    comp_dict = {
                    'year': None,
                    'compare_occu':None,
                    'compare_income':None}
                    
                    if len(occu_idx_list)>0:
                        if event_year < event_dict['first_occu_year']:
                            comp_dict['year'] = int(event_year)
                            comp_dict['compare_occu'] = 'before'
                        elif event_year > event_dict['last_occu_year']:
                            comp_dict['year'] = int(event_year)
                            comp_dict['compare_occu'] = 'after'
                        else:
                            comp_dict['year'] = int(event_year)
                            comp_dict['compare_occu'] = 'middle'
                    if len(income_idx_list)>0:
                        if event_year < event_dict['first_income_year']:
                            comp_dict['year'] = int(event_year)
                            comp_dict['compare_income'] = 'before'
                        elif event_year > event_dict['last_income_year']:
                            comp_dict['year'] = int(event_year)
                            comp_dict['compare_income'] = 'after'
                        else:
                            comp_dict['year'] = int(event_year)
                            comp_dict['compare_income'] = 'middle'
                            
                    comp_list.append(comp_dict)       
                    
                event_dict[event]['happen_time'] = comp_list

        
        event_dict['event_n'] = total_num
        event_vs_career_list.append(event_dict)           

            

In [35]:
vs_list = ['before', 'middle', 'after']
count_dict = {
    'compare_occu':pd.DataFrame(index = event_name_list, columns=['before', 'middle', 'after']),
    'compare_income':pd.DataFrame(index = event_name_list, columns=['before', 'middle', 'after'])
}


for event in event_name_list:
    for time in vs_list:
        n_dict = {
        'compare_occu':0,
        'compare_income':0
        }
        for person in event_vs_career_list:
            if person[event]['happen_time']!=None:
                df = pd.DataFrame.from_dict(person[event]['happen_time'])
                for comp in df.columns[1:]:
                    n_dict[comp] = list(df[comp]).count(time)+ n_dict[comp]
            
        count_dict['compare_occu'][time][event] = n_dict['compare_occu']
        count_dict['compare_income'][time][event] = n_dict['compare_income']
            

In [36]:
count_dict

{'compare_occu':                              before middle after
 marriage_status_change          119   7584   134
 childbearing_status             172   4399     7
 child_under_18_status_change     54   3341   143
 migration_status                 37   1769    24,
 'compare_income':                              before middle after
 marriage_status_change          164   6931   709
 childbearing_status             408   4018   124
 child_under_18_status_change     67   3101   361
 migration_status                 57   1609   160}

In [86]:
event_vs_career_list[1]['child_under_18_status_change']['happen_time']

In [37]:
have_1st_job_list = event_vs_career_list
with open("./NLSW_event.json", 'w') as f:
    json.dump(have_1st_job_list,f, indent = 2, sort_keys=True)

In [38]:
# event number >=1
len([i for i in have_1st_job_list if i['event_n']>0])

4040

In [39]:
# event number >=2
len([i for i in have_1st_job_list if i['event_n']>1])

3286

In [40]:
# event number >=3
len([i for i in have_1st_job_list if i['event_n']>2 ])

2754

In [41]:
## occupation/income每年都有有效值
## 得到 person_id
occu_full_id_list = []
income_full_id_list = []
for idx, person in enumerate(people_dict):
    df = pd.DataFrame.from_dict(person['life_sequence']).replace({np.nan: None})
    occu_sequence = list(df['occupation'])
    if None not in occu_sequence:
        occu_full_id_list.append(person['person_id'])
    income_sequence = df['income']
    if None not in list(income_sequence):
        income_full_id_list.append(person['person_id'])
  
len(occu_full_id_list),len(income_full_id_list)

(564, 200)

In [42]:
##验证
df1=job_info[inc_code_list].replace([-5,-4,-3,-2,-1,0], np.nan)
df1.dropna(axis=0,how='any').shape[0]

200

In [45]:
## occu + 1/2/3
# event number >=1
len([i for i in have_1st_job_list if i['event_n']>0 and i['person_id'] in occu_full_id_list]) #530
# event number >=2
len([i for i in have_1st_job_list if i['event_n']>1 and i['person_id'] in occu_full_id_list]) #504
# event number >=3
len([i for i in have_1st_job_list if i['event_n']>2 and i['person_id'] in occu_full_id_list ]) # 463

530

In [49]:
## income + 1/2/3
# event number >=1
len([i for i in have_1st_job_list if i['event_n']>0 and i['person_id'] in income_full_id_list]) #184
# event number >=2
len([i for i in have_1st_job_list if i['event_n']>1 and i['person_id'] in income_full_id_list]) #176
# event number >=3
len([i for i in have_1st_job_list if i['event_n']>2 and i['person_id'] in income_full_id_list ]) # 157

184